# CNN

In [14]:
# -*- coding: UTF-8 -*-
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt 
from tensorflow.examples.tutorials.mnist import input_data

下载训练数据，55000张图像 （28 \* 28 \* 1）

In [15]:
# help(input_data.read_data_sets)
mnist = input_data.read_data_sets('../data/mnist_data',  one_hot=True)

Extracting ../data/mnist_data/train-images-idx3-ubyte.gz
Extracting ../data/mnist_data/train-labels-idx1-ubyte.gz
Extracting ../data/mnist_data/t10k-images-idx3-ubyte.gz
Extracting ../data/mnist_data/t10k-labels-idx1-ubyte.gz


![image](../images/001.png)

In [16]:
input_x = tf.placeholder(tf.float32, [None, 28*28])/255.
output_y = tf.placeholder(tf.int32, [None, 10])

input_x_images = tf.reshape(input_x, [-1, 28, 28, 1])

test_x = mnist.test.images[:3000]
test_y = mnist.test.labels[:3000]

# 28*28*32
conv1 = tf.layers.conv2d(inputs=input_x_images, #形状 [28 28 1] 
                         filters=32,            #32个过滤器
                         kernel_size=[5,5],     #过滤器大小  5*5
                         strides=1,             #步长 1
                         padding='same',        #same 表示输出大小不变，需要在外面补零2圈
                         activation=tf.nn.relu
                        ) # 形状 28 * 28 * 32
pool1 = tf.layers.max_pooling2d(
            inputs=conv1,               #形状 [28, 28, 32]
            pool_size=[2,2],            #过滤器大小 [2,2]
            strides=2,
        )  # 形状 14 * 14 * 32

conv2 = tf.layers.conv2d(
             inputs=pool1,          #形状 [14, 14, 32] 
             filters=64,            #64个过滤器
             kernel_size=[5,5],     #过滤器大小  5*5
             strides=1,             #步长 1
             padding='same',        #same 表示输出大小不变，需要在外面补零2圈
             activation=tf.nn.relu
        ) #形状[14, 14, 64]

pool2 = tf.layers.max_pooling2d(
            inputs=conv2,               #形状 [14, 14, 64]
            pool_size=[2,2],            #过滤器大小 [2,2]
            strides=2,
        )  # 形状 7 * 7  * 64

flat = tf.reshape(pool2, [-1, 7 *7 * 64]) #形状[7*7pm *64,]

# 1024 全连接层
dense = tf.layers.dense(inputs=flat, units=1024, activation=tf.nn.relu)


dropout = tf.layers.dropout(inputs=dense, rate=0.5, training=True)

logits = tf.layers.dense(inputs=dropout, units=10)

loss = tf.losses.softmax_cross_entropy(
            onehot_labels=output_y, 
            logits=logits)

train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)


accuracy = tf.metrics.accuracy(
    labels=tf.argmax(output_y, axis=1),
    predictions=tf.argmax(logits, axis=1),)[1]




In [18]:
# 创建会话
sess = tf.Session()
# 初始化变量：全局和局部
init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
sess.run(init)

# 训练 5000 步。这个步数可以调节
for i in range(20000):
    batch = mnist.train.next_batch(50)  # 从 Train（训练）数据集里取 “下一个” 50 个样本
    train_loss, train_op_ = sess.run([loss, train_op], {input_x: batch[0], output_y: batch[1]})
    if i % 100 == 0:
        test_accuracy = sess.run(accuracy, {input_x: test_x, output_y: test_y})
        print("第 {} 步的 训练损失={:.4f}, 测试精度={:.2f}".format(i, train_loss, test_accuracy))

# 测试：打印 20 个预测值 和 真实值
test_output = sess.run(logits, {input_x: test_x[:20]})
inferred_y = np.argmax(test_output, 1)
print(inferred_y, '推测的数字')  # 推测的数字
print(np.argmax(test_y[:20], 1), '真实的数字')  # 真实的数字

# 关闭会话
sess.close()

第 0 步的 训练损失=2.3078, 测试精度=0.22
第 100 步的 训练损失=0.1218, 测试精度=0.57
第 200 步的 训练损失=0.1890, 测试精度=0.70
第 300 步的 训练损失=0.1207, 测试精度=0.77
第 400 步的 训练损失=0.0698, 测试精度=0.81
第 500 步的 训练损失=0.0289, 测试精度=0.83
第 600 步的 训练损失=0.0270, 测试精度=0.85
第 700 步的 训练损失=0.1185, 测试精度=0.87
第 800 步的 训练损失=0.2239, 测试精度=0.88
第 900 步的 训练损失=0.1112, 测试精度=0.89
第 1000 步的 训练损失=0.0424, 测试精度=0.90
第 1100 步的 训练损失=0.0050, 测试精度=0.90
第 1200 步的 训练损失=0.0824, 测试精度=0.91
第 1300 步的 训练损失=0.0927, 测试精度=0.91
第 1400 步的 训练损失=0.2378, 测试精度=0.92
第 1500 步的 训练损失=0.0386, 测试精度=0.92
第 1600 步的 训练损失=0.0486, 测试精度=0.93
第 1700 步的 训练损失=0.0461, 测试精度=0.93
第 1800 步的 训练损失=0.0062, 测试精度=0.93
第 1900 步的 训练损失=0.0046, 测试精度=0.93
第 2000 步的 训练损失=0.0270, 测试精度=0.94
第 2100 步的 训练损失=0.0126, 测试精度=0.94
第 2200 步的 训练损失=0.0245, 测试精度=0.94
第 2300 步的 训练损失=0.0377, 测试精度=0.94
第 2400 步的 训练损失=0.0952, 测试精度=0.94
第 2500 步的 训练损失=0.1010, 测试精度=0.94
第 2600 步的 训练损失=0.0336, 测试精度=0.95
第 2700 步的 训练损失=0.0092, 测试精度=0.95
第 2800 步的 训练损失=0.0060, 测试精度=0.95
第 2900 步的 训练损失=0.0770, 测试精度=0.95
第 3000 步的 训练损失=0.1115,